## Passo 2 - Coletando os dados de um episódio

In [1]:
import re

from bs4 import BeautifulSoup
from requests import get

from src.data.make_dataset import ColetorLinksPizza, parse_pagina

Usando o coletor de links para juntar os links dos episódios

In [3]:
c = ColetorLinksPizza()
c.arrasta()

c.links

['https://podcast.pizzadedados.com/e/episodio-023/',
 'https://podcast.pizzadedados.com/e/episodio-022/',
 'https://podcast.pizzadedados.com/e/episodio-021/',
 'https://podcast.pizzadedados.com/e/episodio-020/',
 'https://podcast.pizzadedados.com/e/episodio-019/',
 'https://podcast.pizzadedados.com/e/episodio-018/',
 'https://podcast.pizzadedados.com/e/episodio-017/',
 'https://podcast.pizzadedados.com/e/episodio-016/',
 'https://podcast.pizzadedados.com/e/episodio-015-pizza-de-parabens/',
 'https://podcast.pizzadedados.com/e/episodio-014/',
 'https://podcast.pizzadedados.com/e/a-beleza-matematica-dos-grafos/',
 'https://podcast.pizzadedados.com/e/episodio-012-big-data/',
 'https://podcast.pizzadedados.com/e/episodio-011-bots-politicos/',
 'https://podcast.pizzadedados.com/e/episodio-010-dados-abertos-open-source-e-muito-mais/',
 'https://podcast.pizzadedados.com/e/episodio-009-sobre-crawlers-e-scrapers/',
 'https://podcast.pizzadedados.com/e/episodio-008-conversando-com-bots/',
 'http

# Colocando os dados de um episódio em um dicionário

Pegando a página de um episodio 

In [4]:
link = c.links[0]
sopa = parse_pagina(link)

O primeiro `h2` do episódio é o título do mesmo

In [4]:
d = {
    'episódio': sopa.find_all('h2')[0].text,
}

Já vou colocar também o link direto do episódio

In [5]:
d['link_episodio'] = link

Existem algumas listas em cada episódio o intervalo de `[1:4]` corresponde respectivamente à:

1 - tópicos discutidos no episódio

2 - a lista de assuntos técnicos mencionados (links)

3 - as pérolas do episódio

então vou separar essas listas no objeto `tags`

In [6]:
tags = sopa.find_all('ul')[1:4]

d['assuntos'] = []
d['tópicos'] = []
d['pérolas'] =  []

Pegando tópicos

In [7]:
conteudos = tags[0].find_all('li')
for conteudo in conteudos:
    d['tópicos'].append(conteudo.text)

Pegando assuntos

In [8]:
conteudos = tags[1].find_all('a')
for conteudo in conteudos:
    d['assuntos'].append((conteudo.text, conteudo.attrs['href']))

Pegando pérolas

In [9]:
conteudos = tags[2].find_all('li')
for conteudo in conteudos:
    d['pérolas'].append(conteudo.text)

Um jeito fácil de saber quantos convidados temos nos episódios é pegar todos os participantes, facilmente identificaveis pela tag `img` que tem atributo de `style` com o conteudo `"border-radius: 50%;"` e subtrair 3 (quantidade de pizzaiolos em todos os episódios)

In [10]:
d['num_participantes'] = len(sopa.find_all('img', style=re.compile("border-radius: 50%;")))
d['num_convidados'] = d['num_participantes'] - 3

Eu quero coletar também os nomes dos convidados e participantes, os nomes estão sempre dentro da tag `h3`, com já sabemos a quantidade de partipantes podemos usar o `find_all` e filtrar pela quantidade.

In [11]:
d['participantes'] = []
conteudos = sopa.find_all('h3')[:d['num_participantes']]
for conteudo in conteudos:
    d['participantes'].append(conteudo.text)

In [12]:
d['convidados'] = []
for conteudo in conteudos[-d['num_convidados']:]:
    d['convidados'].append(conteudo.text)

In [13]:
d

{'episódio': 'Episódio 023: BioinfoRmática',
 'link_episodio': 'https://podcast.pizzadedados.com/e/episodio-023/',
 'assuntos': [('#PerguntaProPizza',
   'https://twitter.com/search?q=%23perguntapropizza&src=typd'),
  ('Instagram do pizza', 'https://www.instagram.com/pizzadedados/'),
  ('Apoia-se do Pizza', 'https://apoia.se/pizzadedados'),
  ('Canal do Pizza no Telegram', 'https://t.me/pizzadedados'),
  ('Artigo do Adam Drake sobre command line tools',
   'https://adamdrake.com/command-line-tools-can-be-235x-faster-than-your-hadoop-cluster.html'),
  ('Principles and techniques of data science - Universidade de Berkley',
   'https://adamdrake.com/command-line-tools-can-be-235x-faster-than-your-hadoop-cluster.html'),
  ('Tensorflow #1 na Revista do Pizza',
   'https://adamdrake.com/command-line-tools-can-be-235x-faster-than-your-hadoop-cluster.html'),
  ('Tensorflow #2 na Revista do Pizza',
   'https://adamdrake.com/command-line-tools-can-be-235x-faster-than-your-hadoop-cluster.html'),


___
# Criando funções

In [1]:
import re
from src.data.make_dataset import parse_pagina


link = 'https://podcast.pizzadedados.com/e/episodio-023/'
sopa = parse_pagina(link)
novo_d = {
    'episódio': sopa.find_all('h2')[0].text,
    'link_episodio': link,
    'assuntos': [],
    'tópicos': [],
    'pérolas':  [],
    'num_participantes': 0,
    'num_convidados': 0,
    'nomes_participantes': [],
    'nomes_convidados': [],
}

___
## Contando os participantes

In [2]:
def conta_participantes(sopa):
    return len(sopa.find_all('img', style=re.compile("border-radius: 50%;")))

In [3]:
num_part = conta_participantes(sopa)
novo_d['num_participantes'] = num_part
novo_d['num_convidados'] = num_part - 3 

In [4]:
novo_d

{'episódio': 'Episódio 023: BioinfoRmática',
 'link_episodio': 'https://podcast.pizzadedados.com/e/episodio-023/',
 'assuntos': [],
 'tópicos': [],
 'pérolas': [],
 'num_participantes': 4,
 'num_convidados': 1,
 'nomes_participantes': [],
 'nomes_convidados': []}

___
## Buscando conteudos
encontrando topicos, participantes, convidados e perolas

In [7]:
def busca_conteudos(tag, html_tag, lim_max=0, lim_min=0):
    if lim_max and lim_min:
        conteudos = tag.find_all(html_tag)[lim_min:lim_max]
        return conteudos
    if lim_max:
        conteudos = tag.find_all(html_tag)[:lim_max]
        return conteudos
    conteudos = tag.find_all(html_tag)
    return conteudos

Usando a `busca_conteudos` para encontrar as tags com topicos, perolas, participantes e convidados

In [8]:
tags = busca_conteudos(tag=sopa, html_tag='ul', lim_max=4, lim_min=1)
len(tags)

3

Usando a `busca_conteudos` para encontrar as tags com topicos

In [9]:
contents = busca_conteudos(tags[0], 'li')
contents

[<li style="font-weight: 400;"><span style="font-weight: 400;">Quem é e o que faz a nossa convidada?</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">O que uma bioinformata faz em um hospital?</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Como usar análise de dados para ajudar pacientes de câncer</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Quais são as dificuldades de analisar dados biológicos?</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Vamos falar de R!</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Pesquisa acadêmica Estados Unidos vs Brasil</span></li>]

Usando a `busca_conteudos` para encontrar as tags com perolas

In [10]:
contents = busca_conteudos(tags[2], 'li')
contents

[<li style="font-weight: 400;"><span style="font-weight: 400;">Eu gosto de fazer promessas que não necessariamente vão ser cumpridas</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Pessoal que está morando lá há um tempo fala certinho… acho mó chique</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Saudades inclusive</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Vou parar de falar pra não falar muitos nomes esquisitos</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Agora eu vou rasgar seda pra comunidade de R</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Jessica tá feliz que a gente está fazendo um episódio de R</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 400;">Dado aberto é pizza e dado fechado é calzone. E a gente gosta de pizza e não de calzone.</span></li>,
 <li style="font-weight: 400;"><span style="font-weight: 40

Usando a `busca_conteudos` para encontrar tags com os participantes do episodio

In [11]:
maximo = novo_d['num_participantes']
participantes = busca_conteudos(sopa, 'h3', lim_max=maximo)
participantes

[<h3>Leticia Portella</h3>,
 <h3>Jessica Temporal</h3>,
 <h3>Gustavo Coelho</h3>,
 <h3>Thais Sarraf Sabedot</h3>]

In [12]:
minimo = novo_d['num_participantes'] - novo_d['num_convidados']
convidados = busca_conteudos(sopa, 'h3', lim_min=minimo, lim_max=maximo)
convidados

[<h3>Thais Sarraf Sabedot</h3>]

___
## Recuperando o texto de uma tag

Eu posso usar o método `__getattribute__` e passar uma string com o nome do atributo que quero como `'text'` e com isso montar uma função para buscar o texto de uma tag

In [13]:
def busca_texto(conteudo):
    """Método que retorna o texto de uma tag
    
    Parâmetros::
    
    :param conteudo: [bs4.Tag], uma lista de tags do tipo `li`
    
    Retornos::
    
    :returns: lista de strings
    :rtype: list
    
    Uso::
    
      >>> busca_texto(tag)
    """
    return conteudo.__getattribute__('text')

In [14]:
busca_texto(convidados[0])

'Thais Sarraf Sabedot'

___
## Pegando o texto e link de um topico

Eu posso usar o método `__getattribute__` em conjunto com método `__getitem__` para obter o texto e o link de uma tag do tipo `a`

In [15]:
def busca_href(conteudo):
    """Método que retorna o texto de uma tag `a` e o seu conteudo textual
    
    Parâmetros::
    
    :param conteudo: bs4.Tag, uma tag do tipo `a`
    
    Retornos::
    
    :returns: uma tupla contendo o texto e o link da tag
    :rtype: tuple
    
    Uso::
    
      >>> busca_href(tag)
    """
    return (conteudo.__getattribute__('text'),
            conteudo.__getattribute__('attrs').__getitem__('href'))

In [16]:
contents = busca_conteudos(tags[1], 'a')
busca_href(contents[1])

('Instagram do pizza', 'https://www.instagram.com/pizzadedados/')

___
## Pegando texto de um conjunto de tags

Eu posso usar o método `__getattribute__` e passar uma string com o nome do atributo que quero como `'text'` e com isso montar uma função para buscar o texto de uma tag

In [20]:
def busca_textos(conteudos):
    """Método que retornar uma lista de textos contidos em uma lista de tags
    
    Parâmetros::
    
    :param conteudos: [bs4.Tag], uma lista de tags
    
    Retornos::
    
    :returns: lista de strings
    :rtype: list
    
    Uso::
    
      >>> busca_itens(conteudos)
    """
    itens = []
    for conteudo in conteudos:
        itens.append(conteudo.__getattribute__('text'))
    return itens

In [21]:
busca_textos(participantes)

['Leticia Portella',
 'Jessica Temporal',
 'Gustavo Coelho',
 'Thais Sarraf Sabedot']